In [36]:
from datasets import load_from_disk
datasets = load_from_disk("CV11_distilgpt2_3U_0D_PREPROC_100%.hf")
print(datasets["train"][0])

{'input_ids': [5239, 25, 2750, 428, 640, 11, 449, 648, 338, 1438, 550, 1716, 1900, 6768, 3690, 7229, 13, 198, 23661, 25, 220, 50319, 51170, 50665, 50801, 50319, 50681, 50319, 50681, 50665, 50801, 50665, 50801, 50363, 50775, 50399, 50911, 50710, 51171, 51012, 51167, 50733, 50285, 50688, 50895, 50788, 50872, 50946, 50440, 51139, 50984, 51275, 51058, 50710, 50960, 51139, 50443, 50830, 50273, 50326, 50839, 50578, 51132, 51161, 50817, 51091, 51199, 51091, 51000, 51091, 51027, 51112, 50661, 50320, 51217, 50768, 51132, 51175, 50669, 50779, 50629, 50784, 51067, 50779, 50899, 50734, 50274, 50873, 51167, 50939, 50831, 51206, 50440, 50630, 51132, 50560, 50887, 50560, 51132, 50560, 51132, 50560, 50788, 50560, 51280, 50560, 50433, 50560, 50930, 51233, 50433, 51117, 51177, 51076, 50936, 51112, 51153, 50433, 51230, 50831, 51167, 50321, 50702, 50360, 50728, 50985, 51204, 50744, 50526, 50817, 51187, 50479, 50872, 50837, 50670, 50788, 50499, 50688, 50574, 50496, 50909, 50788, 51187, 50830, 50293, 50902,

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from encodec import EncodecModel
from encodec.utils import convert_audio
import torch
import torchaudio
import os
import re
from tqdm import tqdm

In [3]:
encodec_model = EncodecModel.encodec_model_24khz()
encodec_model.set_target_bandwidth(1.5)
model_name = "anforsm/distilgpt2-finetuned-common-voice"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

c:\Python38\lib\site-packages\huggingface_hub\file_download.py:129: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [33]:
def decode(tokens):
    decoded = tokenizer.decode(tokens[0], skip_special_tokens=True)
    # Get all audio_token_
    pattern = r'audio_token_(\d+)'
    audio_tokens = re.findall(pattern, decoded)
    audio_tokens = [int(token) for token in audio_tokens]
    print(decoded)
    print(len(audio_tokens))
    print(audio_tokens)

    number_of_codebooks = 2
    number_of_samples = len(audio_tokens) // number_of_codebooks
    frame = torch.zeros(1, number_of_codebooks, number_of_samples, dtype=torch.long)
    for sample in range(number_of_samples):
        for codebook in range(number_of_codebooks):
            frame[0, codebook, sample] = audio_tokens[sample * number_of_codebooks + codebook]
    
    frames = [(frame, None)]

    with torch.no_grad():
        wav = encodec_model.decode(frames)

    torchaudio.save("output.wav", wav[0, :, :], encodec_model.sample_rate)

In [9]:
tokens = datasets["train"][3]["input_ids"]



TypeError: expected string or bytes-like object

In [25]:
text = tokenizer.decode(tokens)

pattern = r'audio_token_(\d+)'
audio_tokens = re.findall(pattern, text)
audio_tokens

['62',
 '913',
 '408',
 '544',
 '62',
 '424',
 '62',
 '424',
 '408',
 '544',
 '408',
 '544',
 '408',
 '913',
 '408',
 '913',
 '408',
 '913',
 '408',
 '913',
 '73',
 '648',
 '984',
 '21',
 '429',
 '783',
 '69',
 '413',
 '147',
 '619',
 '147',
 '621',
 '147',
 '621',
 '147',
 '619',
 '147',
 '619',
 '147',
 '619',
 '147',
 '956',
 '124',
 '373',
 '124',
 '956',
 '324',
 '515',
 '324',
 '237',
 '203',
 '225',
 '429',
 '90',
 '749',
 '493',
 '203',
 '98',
 '949',
 '654',
 '984',
 '624',
 '977',
 '654',
 '275',
 '74',
 '20',
 '53',
 '698',
 '872',
 '857',
 '467',
 '253',
 '979',
 '797',
 '795',
 '406',
 '623',
 '312',
 '301',
 '312',
 '195',
 '312',
 '688',
 '328',
 '504',
 '255',
 '942',
 '879',
 '987',
 '312',
 '564',
 '182',
 '785',
 '235',
 '619',
 '280',
 '621',
 '768',
 '926',
 '1018',
 '121',
 '926',
 '469',
 '857',
 '182',
 '23',
 '307',
 '834',
 '282',
 '879',
 '1007',
 '753',
 '419',
 '679',
 '153',
 '724',
 '266',
 '887',
 '928',
 '887',
 '71',
 '491',
 '1023',
 '753',
 '700',
 '

In [35]:
decode([datasets["train"][9]["input_ids"]])

text: He made his public debut as a vocalist at the age of six.
sound: audio_token_62audio_token_913audio_token_408audio_token_544audio_token_408audio_token_544audio_token_62audio_token_424audio_token_408audio_token_544audio_token_408audio_token_518audio_token_408audio_token_518audio_token_408audio_token_518audio_token_25audio_token_652audio_token_463audio_token_974audio_token_463audio_token_974audio_token_463audio_token_974audio_token_463audio_token_364audio_token_463audio_token_364audio_token_25audio_token_652audio_token_25audio_token_364audio_token_432audio_token_870audio_token_311audio_token_985audio_token_612audio_token_2audio_token_967audio_token_527audio_token_457audio_token_349audio_token_868audio_token_868audio_token_967audio_token_53audio_token_457audio_token_632audio_token_1022audio_token_359audio_token_862audio_token_654audio_token_457audio_token_655audio_token_967audio_token_632audio_token_793audio_token_245audio_token_830audio_token_711audio_token_240audio_token_268audio_